In [4]:
#Loading modules
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 

import sklearn
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch import nn, optim
import torch.nn.functional as F

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

print(torch.__version__)
print(sklearn.__version__)
print(np.__version__)
print(matplotlib.__version__)

/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
1.5.0
0.22.2.post1
1.18.1


NameError: name 'matplotlib' is not defined

In [ ]:
#Loading data
trainSet = pd.read_csv("/kaggle/input/digit-recognizer/train.csv", dtype=np.float32)
evaluationTestSet = pd.read_csv("/kaggle/input/digit-recognizer/test.csv", dtype=np.float32)

targetsNp = trainSet.label.values
featuresNp = trainSet.loc[:, trainSet.columns != 'label'].values/255

# Split to training and test set
featuresTrain, featuresTest, targetsTrain, targetsTest = train_test_split(featuresNp, targetsNp, test_size=0.2, random_state=42)


BATCH_SIZE = 100
featuresTrain = torch.from_numpy(featuresTrain)
targetsTrain = torch.from_numpy(targetsTrain).type(torch.LongTensor)
featuresTest = torch.from_numpy(featuresTest)
targetsTest = torch.from_numpy(targetsTest).type(torch.LongTensor)


train = torch.utils.data.TensorDataset(featuresTrain,targetsTrain)
test = torch.utils.data.TensorDataset(featuresTest,targetsTest)

#Data preparation
trainLoader = DataLoader(train, batch_size = BATCH_SIZE, shuffle = True)
testLoader = DataLoader(test, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.dropoutP = 0.15
        self.dropout = nn.Dropout(p=self.dropoutP)
        
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64,32)
        self.fc6 = nn.Linear(32,10)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.dropout(F.relu(self.fc4(x)))
        x = self.dropout(F.relu(self.fc5(x)))
        
        x = F.log_softmax(self.fc6(x), dim = 1)
        
        return x
                         
class CNNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.dropoutP = 0.15
        self.dropout = nn.Dropout(p=self.dropoutP)
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5)
        self.conv1Pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.conv2Pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    def forward(self, x):
        x = F.relu(self.conv1Pool(self.conv1(x)))
        x = F.relu(self.conv2Pool(self.conv2(x)))
        
        x = x.view(-1,320)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        
        x = F.log_softmax(x, dim = 1)
        
        return x



In [ ]:
NETWORK_TYPE = "CNN"

if NETWORK_TYPE == "CNN":
    ClassificationNetwork = CNNet()
if NETWORK_TYPE == "ANN":
    ClassificationNetwork = Net()
    
EPOCHS = 10  
lossFunction = nn.NLLLoss()

LEARNING_RATE = 0.1
optimizer = optim.SGD(ClassificationNetwork.parameters(), lr=LEARNING_RATE)



In [ ]:
trainLosses = []
testLosses = []
testAccuracies = []
steps = []

step = 0
for e in range(EPOCHS):
    runningLoss = 0
    for images, labels in trainLoader:
        step += 1
        
        optimizer.zero_grad()
        
        if NETWORK_TYPE == "CNN":
            images = Variable(images.view(100,1,28,28))
        if NETWORK_TYPE == "ANN":
            images = Variable(images.view(-1, 28*28))
              
        logPredictions = ClassificationNetwork(images)
        loss = lossFunction(logPredictions,labels)
        
        loss.backward()
        optimizer.step()
        
        runningLoss += loss.item()
        
        if step % 20 == 0:
            testLoss = 0
            accuracy = 0

            with torch.no_grad():
                ClassificationNetwork.eval()
                for images, labels in testLoader:
                    
                    if NETWORK_TYPE == "CNN":
                        images = Variable(images.view(100,1,28,28))
                    if NETWORK_TYPE == "ANN":
                        images = Variable(images.view(-1, 28*28))
                    
                    logPrediction = ClassificationNetwork(images)
                    testLoss += lossFunction(logPrediction, labels)

                    # Get our top predictions
                    topPrediction, topClass = logPrediction.topk(1, dim=1)
                    equals = topClass == labels.view(*topClass.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
                    
            ClassificationNetwork.train()

            
            trainLosses.append(runningLoss/len(trainLoader))
            testLosses.append(testLoss/len(testLoader))
            testAccuracies.append(accuracy/len(testLoader))
            steps.append(step)
            
            print(f"--------------------Epoch: {e+1}/{EPOCHS}--------------------\n"+
                 f"Train loss = {trainLosses[-1]:.5f}\t\tTest lost = {testLosses[-1]:.5f}\n" +
                 f"Test Accuracy = {accuracy/len(testLoader):.5f}")
            
                    

In [ ]:

plt.plot(steps, testAccuracies)
plt.xlabel("Iteration step")
plt.ylabel(f"{NETWORK_TYPE} accuracy")
plt.show()

plt.plot(steps,testLosses,color = "red")
plt.xlabel("Iteration step")
plt.ylabel(f"{NETWORK_TYPE} loss")
plt.show()